<a href="https://colab.research.google.com/github/AluminumShark/2024_Deep_Learning_with_Pytorch/blob/main/01_MNIST_ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Import Modules

import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
from torchvision.datasets import MNIST
from torch.utils.data import DataLoader
import numpy as np
torch.manual_seed(681520)

In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
EPOCHS = 20
BATCH_SIZE = 128
lr = 0.001

In [3]:
# Load and Normalize Datasets

trainset = MNIST(root='./data', train=True, transform=transforms.ToTensor(), download=True)
train_loader = DataLoader(dataset=trainset, batch_size=len(trainset), shuffle=True)

data = next(iter(train_loader))

mean = data[0].mean()
std = data[0].std()

transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize(mean, std)])

trainset = MNIST(root='./data', train=True, transform=transform, download=True)
testset = MNIST(root='./data', train=True, transform=transform, download=True)

Failed to download (trying next):
<urlopen error [SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: certificate has expired (_ssl.c:1007)>



100%|██████████| 9.91M/9.91M [00:00<00:00, 39.7MB/s]


Extracting ./data/MNIST/raw/train-images-idx3-ubyte.gz to ./data/MNIST/raw

Failed to download (trying next):
<urlopen error [SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: certificate has expired (_ssl.c:1007)>



100%|██████████| 28.9k/28.9k [00:00<00:00, 1.26MB/s]

Extracting ./data/MNIST/raw/train-labels-idx1-ubyte.gz to ./data/MNIST/raw



Failed to download (trying next):
<urlopen error [SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: certificate has expired (_ssl.c:1007)>



100%|██████████| 1.65M/1.65M [00:00<00:00, 8.45MB/s]


Extracting ./data/MNIST/raw/t10k-images-idx3-ubyte.gz to ./data/MNIST/raw

Failed to download (trying next):
<urlopen error [SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: certificate has expired (_ssl.c:1007)>



100%|██████████| 4.54k/4.54k [00:00<00:00, 7.90MB/s]


Extracting ./data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/MNIST/raw



In [8]:
# Creat a Sequential Model

model = nn.Sequential(
    nn.Flatten(),

    nn.Linear(28*28, 25),
    nn.BatchNorm1d(25),
    nn.ReLU(),

    nn.Linear(25, 10),

)

for module in model.modules():
    if isinstance(module, nn.Linear):
        nn.init.kaiming_uniform_(module.weight, nonlinearity='relu')
        nn.init.constant_(module.bias, 0)

In [9]:
optimizer = torch.optim.Adam(model.parameters(), lr)
loss_fn = nn.CrossEntropyLoss()

model.to(device)

train_loader = DataLoader(trainset, batch_size=BATCH_SIZE, shuffle=True)
test_loader = DataLoader(testset, batch_size=BATCH_SIZE, shuffle=True)


for i in range(EPOCHS):
  model.train()
  train_loss = 0.0
  train_acc = 0
  train_batches = 0

  # Start Training

  for inputs, targets in train_loader:
    one_hot_targets = nn.functional.one_hot(targets, num_classes=10).float()
    inputs, targets, one_hot_targets = inputs.to(device), targets.to(device), one_hot_targets.to(device)

    optimizer.zero_grad()

    # Forward pass
    outputs = model(inputs)
    loss = loss_fn(outputs, one_hot_targets)

    # Caculate Accuracy
    _, predicted = torch.max(outputs.data, 1)
    train_acc += (predicted == targets).sum().item()
    train_batches += 1
    train_loss += loss.item()

    # Backward pass and Update
    loss.backward()
    optimizer.step()

  train_loss /= train_batches
  train_acc /= train_batches*BATCH_SIZE

  # Evaluate Model

  model.eval()
  test_loss = 0.0
  test_acc = 0
  test_batches = 0

  for inputs, targets in test_loader:
    one_hot_targets = nn.functional.one_hot(targets, num_classes=10).float()
    inputs, targets, one_hot_targets = inputs.to(device), targets.to(device), one_hot_targets.to(device)

    # Forward pass
    outputs = model(inputs)
    loss = loss_fn(outputs, one_hot_targets)

    # Calculate Accuracy
    _, predicted = torch.max(outputs.data, 1)
    test_acc += (predicted == targets).sum().item()
    test_batches += 1
    test_loss += loss.item()

  test_loss /= test_batches
  test_acc /= test_batches*BATCH_SIZE

  print(f'Epoch {i+1}/{EPOCHS} loss: {train_loss:.4f} - acc: {train_acc:0.4f} - val_loss: {test_loss:.4f} - val_acc: {test_acc:0.4f}')

Epoch 1/20 loss: 0.5947 - acc: 0.8505 - val_loss: 0.2918 - val_acc: 0.9251
Epoch 2/20 loss: 0.2592 - acc: 0.9299 - val_loss: 0.2108 - val_acc: 0.9423
Epoch 3/20 loss: 0.2053 - acc: 0.9421 - val_loss: 0.1715 - val_acc: 0.9524
Epoch 4/20 loss: 0.1769 - acc: 0.9501 - val_loss: 0.1481 - val_acc: 0.9588
Epoch 5/20 loss: 0.1572 - acc: 0.9552 - val_loss: 0.1311 - val_acc: 0.9640
Epoch 6/20 loss: 0.1440 - acc: 0.9586 - val_loss: 0.1208 - val_acc: 0.9663
Epoch 7/20 loss: 0.1335 - acc: 0.9612 - val_loss: 0.1092 - val_acc: 0.9698
Epoch 8/20 loss: 0.1246 - acc: 0.9633 - val_loss: 0.1014 - val_acc: 0.9717
Epoch 9/20 loss: 0.1167 - acc: 0.9660 - val_loss: 0.0957 - val_acc: 0.9736
Epoch 10/20 loss: 0.1108 - acc: 0.9669 - val_loss: 0.0902 - val_acc: 0.9739
Epoch 11/20 loss: 0.1054 - acc: 0.9688 - val_loss: 0.0867 - val_acc: 0.9753
Epoch 12/20 loss: 0.1002 - acc: 0.9700 - val_loss: 0.0795 - val_acc: 0.9772
Epoch 13/20 loss: 0.0960 - acc: 0.9707 - val_loss: 0.0755 - val_acc: 0.9786
Epoch 14/20 loss: 0.0